# Customizing Lens

Lens strives to give you sensible defaults and automatically do the proper assessments whenever possible. However, there are many times where you'll want to change, select, or extend the functionality. Lens is intended to be an _extensible framework_ that can accomodate your own analysis plan.

In this document we will describe a number of ways you can customize Lens:
* Setting up CredoModels
* Selecting which assessments to run
* Parameterizing assessments
* Creating new modules and assessments
* Incorporating new metrics

In [1]:
import credoai.lens as cl
from copy import deepcopy



Failed to import tensorflow. Please note that tensorflow is not installed by default when you install tensorflow_hub. This is so that users can decide which tensorflow package to use. To use tensorflow_hub, please install a current version of tensorflow by following the instructions at https://tensorflow.org/install and https://tensorflow.org/hub/installation.




## Setup

We'll set up data and a model to run through some customization options. See `quickstart` for more information on this code.

In [2]:
# imports for example data and model training
from credoai.data import fetch_creditdefault
from sklearn.ensemble import GradientBoostingClassifier
# data
data = fetch_creditdefault(as_frame=True)
X = data['data'].drop(columns=['SEX'])
y = data['target']
sensitive_feature = data['data']['SEX']
# model
model = GradientBoostingClassifier()
model.fit(X,y)

GradientBoostingClassifier()

In [3]:
credo_model = cl.CredoModel(name='credit_default_classifier',
                            model=model)

credo_data = cl.CredoData(name='UCI-credit-default',
                          X=X, 
                          y=y.astype(int),
                          sensitive_features=sensitive_feature)
alignment_spec = {'FairnessBase': {'metrics': ['precision_score']}}

## Creating a CredoModel

Above the CredoModel functionality was inferred from the fact that the model (GraidentBoostingClassifier) is a scikit-learn model. But CredoModels are generic wrappers around functions and can be created from scratch.

The below "custom_model" instantiates two custom functions: `foo` and `bar`. Specific functionality is required by different assessments (see below). For instance, some assessments require that the CredoModel instantiates a `prob_fun`.

Bottom line - CredoModels instantiate certain functionality, and this functionality determines which assessments can be run successfully.

In [4]:
model_config = {'foo': lambda: 'foo response', 'bar': lambda: 'bar response'}
custom_model = cl.CredoModel(name = 'example',
                             model_config = model_config)

custom_model.foo()

'foo response'

## Selecting Assessments

**Automatic selection of assessments**

Lens has a number of assessments available, each of which works with different kinds of models or datasets. By default, Lens will automatically run every assessment that has its prerequesites met. The prerequesities can be queried by calling the `get_requirements` function on an assessment. These indicate the set of features or functions your model and data must instantiate in order for the assessment to be run. 

These requirements are specific for the model and the data. Each requirement is either a single requirements or a tuple. If a tuple, only one of the requirements within the tuple must be met. For instance, the FairnessBaseAssessment needs *either* `prob_fun` OR `pred_fun`. See `credoai.assessments.credo_assessment.AssessmentRequirements` for more.

In [5]:
from credoai.assessment import FairnessBaseAssessment
assessment = FairnessBaseAssessment()
assessment.get_requirements()

{'model_requirements': [('prob_fun', 'pred_fun')],
 'data_requirements': ['X', 'y', 'sensitive_features']}

You can also get the requirements for all assessments.

In [6]:
from credoai.assessment import get_assessment_requirements
get_assessment_requirements()

{'DatasetAssessment': {'model_requirements': [],
  'data_requirements': ['X', 'y', 'sensitive_features']},
 'FairnessBaseAssessment': {'model_requirements': [('prob_fun', 'pred_fun')],
  'data_requirements': ['X', 'y', 'sensitive_features']},
 'NLPEmbeddingBiasAssessment': {'model_requirements': ['embedding_fun'],
  'data_requirements': []},
 'NLPGeneratorAssessment': {'model_requirements': ['generation_fun',
   'assessment_config'],
  'data_requirements': []}}

**Selecting a subset of assessments**

But what if you don't want all of those assessments? No problem! Just pass the assessments you do want to Lens. 

In [7]:
# Note: Remember to instantiate the assessment!
selected_assessments = [FairnessBaseAssessment()]
lens = cl.Lens(model=credo_model,
               data=credo_data,
               assessments=selected_assessments,
               spec=alignment_spec)

## Parameterizing Assessments

Now that we can select assessments, how about customizing them? There are two places where assessments can be customized: (1) when their underlying module is initialized and (2) when they are ran (which runs the underlying module).

### Customizing initialization
Below we can see how to customize the initialization of the assessment. We use something we've already seen before - the `alignment spec`! The parameters that can be passed at this stage are the same parameters passed to the assessment's `init_module` function.

In [8]:
# Note: Remember to instantiate the assessment!
selected_assessments = [FairnessBaseAssessment()]
init_kwargs = deepcopy(alignment_spec)
init_kwargs['FairnessBase']['metrics'].append('recall_score')
lens = cl.Lens(model=credo_model,
               data=credo_data,
               assessments=selected_assessments,
               spec=init_kwargs)

In [9]:
lens.run_assessments()['FairnessBase']['fairness']

,value
precision_score,0.000543
recall_score,0.01676


### Customizing running assessments 

The other way of parameterizing the assessments is by passing arguments to the assessment's `run` function. These kwargs are passed to `lens.run_assessments`, which are, in turn passed to the assessment's initialized module.

For instance, the `FairnessBase` assessment initializes `mod.FairnessBase`, whose `run` argument can take a `method` parameter which controls how fairness scores are calculated. The default is "between_groups", but we can change it like so:

In [10]:
run_kwargs = {'FairnessBase': {'method': 'to_overall'}}
lens.run_assessments(assessment_kwargs = run_kwargs)['FairnessBase']['fairness']

,value
precision_score,0.000302
recall_score,0.009504


## Creating New Modules & Assessments

WIP section!

## Module Specific Customization - Custom Metrics for Fairness Base

Each module has different parameterization options, as discused above. The FairnessModule takes a set of metrics to calculate on the model and data. Many metrics are supported out-of-the-box. These metrics can be referenced by string. However, custom metrics can be created as well. Doing so will allow you to calculate any metric that takes in a `y_true` and some kind of prediction

Custom metrics can be incorporated by creating a `FairnessFunction`. The `FairnessFunction` is a lightweight wrapper class that defines a few characteristics of the custom function needed by the module. 

**Example: Confidence Intervals**

We will create custom metrics that reflect the lower and upper 95th percentile confidence bound on the true positive rate.

Confidence intervals are not generically supported. However, they can be derived for metrics derived from a confusion matrix using the `wilson confidence interval`. A convenience function called `confusion_wilson` is supplied which returns an array: [lower, upper] bound for the metric. 

Wrapping the confusion wilson function in a `FairnessFunction` allows us to pass it as a metric to the FairnessModule.

In [15]:
from credoai.utils.metrics import confusion_wilson
from credoai.modules import FairnessFunction, FairnessModule

# define partial functions for the true positive rate lower bound
def lower_bound_tpr(y_true, y_pred):
    return confusion_wilson(y_true, y_pred, metric='true_positive_rate', confidence=0.95)[0]

# and upper bound
def upper_bound_tpr(y_true, y_pred):
    return confusion_wilson(y_true, y_pred, metric='true_positive_rate', confidence=0.95)[1]

# wrap the functions in fairness functions
lower_metric = FairnessFunction(name = 'lower_bound_tpr', 
                          func = lower_bound_tpr, 
                          takes_sensitive_features = False, 
                          takes_prob = False)

upper_metric = FairnessFunction(name = 'upper_bound_tpr', 
                          func = upper_bound_tpr, 
                          takes_sensitive_features = False, 
                          takes_prob = False)

In [19]:
# Note: Remember to instantiate the assessment!
selected_assessments = [FairnessBaseAssessment()]
init_kwargs = deepcopy(alignment_spec)
init_kwargs['FairnessBase']['metrics'] = [lower_metric, 'tpr', upper_metric]
custom_lens = cl.Lens(model=credo_model,
               data=credo_data,
               assessments=selected_assessments,
               spec=init_kwargs)
output = custom_lens.run_assessments()
output['FairnessBase']['disaggregated_results']

,lower_bound_tpr,tpr,upper_bound_tpr
SEX,,,
1.0,0.365613,0.383223,0.401145
2.0,0.351211,0.366463,0.381987
overall,0.362155,0.373719,0.385429
